<a href="https://colab.research.google.com/github/KNGLJordan/CDMO-project/blob/main/src/SAT/sat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt install python3-z3

[sudo] password for lucazini03: 


In [1]:
from z3 import *
from itertools import combinations

In [3]:
#the man the myth the legend: angelo quarta

from z3 import *
from itertools import combinations

#NAIVE PAIRWISE
def at_least_one_np(bool_vars):
    return Or(bool_vars)

def exactly_one_np(bool_vars, name=""):  # Already OK
    return And(at_least_one_np(bool_vars), at_most_one_np(bool_vars, name))

def at_most_one_np(bool_vars, name=""):  # Already OK
    return And([Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)])

def at_least_k_np(bool_vars, k, name=""):  # <- add `name`
    return at_most_k_np([Not(var) for var in bool_vars], len(bool_vars) - k, name)

def at_most_k_np(bool_vars, k, name=""):  # <- add `name`
    return And([Or([Not(x) for x in X]) for X in combinations(bool_vars, k + 1)])

def exactly_k_np(bool_vars, k, name=""):  # <- add `name`
    return And(at_most_k_np(bool_vars, k, name), at_least_k_np(bool_vars, k, name))



#SEQUENTIAL
def at_least_one_seq(bool_vars=''):
    return at_least_one_np(bool_vars)

def at_most_one_seq(bool_vars, name=''):
    constraints = []
    n = len(bool_vars)
    s = [Bool(f"s_{name}_{i}") for i in range(n - 1)]
    constraints.append(Or(Not(bool_vars[0]), s[0]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2])))
    for i in range(1, n - 1):
        constraints.append(Or(Not(bool_vars[i]), s[i]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1])))
        constraints.append(Or(Not(s[i-1]), s[i]))
    return And(constraints)

def exactly_one_seq(bool_vars, name=''):
    return And(at_least_one_seq(bool_vars), at_most_one_seq(bool_vars, name))

def at_least_k_seq(bool_vars, k, name=''):
    return at_most_k_seq([Not(var) for var in bool_vars], len(bool_vars)-k, name)

def at_most_k_seq(bool_vars, k, name=''):
    constraints = []
    n = len(bool_vars)
    s = [[Bool(f"s_{name}_{i}_{j}") for j in range(k)] for i in range(n - 1)]
    constraints.append(Or(Not(bool_vars[0]), s[0][0]))
    constraints += [Not(s[0][j]) for j in range(1, k)]
    for i in range(1, n-1):
        constraints.append(Or(Not(bool_vars[i]), s[i][0]))
        constraints.append(Or(Not(s[i-1][0]), s[i][0]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][k-1])))
        for j in range(1, k):
            constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][j-1]), s[i][j]))
            constraints.append(Or(Not(s[i-1][j]), s[i][j]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2][k-1])))
    return And(constraints)

def exactly_k_seq(bool_vars, k, name=''):
    return And(at_most_k_seq(bool_vars, k, name), at_least_k_seq(bool_vars, k, name))

#HEULE
def at_least_one_he(bool_vars, name=''):
    return at_least_one_np(bool_vars)

def at_most_one_he(bool_vars, name=''):
    if len(bool_vars) <= 4:
        return And(at_most_one_np(bool_vars))
    y = Bool(f"y_{name}")
    return And(And(at_most_one_np(bool_vars[:3] + [y])), And(at_most_one_he(bool_vars[3:] + [Not(y)], name+"_")))

def exactly_one_he(bool_vars, name=''):
    return And(at_most_one_he(bool_vars, name), at_least_one_he(bool_vars))

#BINARY
def toBinary(num, length = None):
    num_bin = bin(num).split("b")[-1]
    if length:
        return "0"*(length - len(num_bin)) + num_bin
    return num_bin
    
def at_least_one_bw(bool_vars, name=''):
    return at_least_one_np(bool_vars)

def at_most_one_bw(bool_vars, name):
    constraints = []
    n = len(bool_vars)
    m = math.ceil(math.log2(n))
    r = [Bool(f"r_{name}_{i}") for i in range(m)]
    binaries = [toBinary(i, m) for i in range(n)]
    for i in range(n):
        for j in range(m):
            phi = Not(r[j])
            if binaries[i][j] == "1":
                phi = r[j]
            constraints.append(Or(Not(bool_vars[i]), phi))        
    return And(constraints)

def exactly_one_bw(bool_vars, name):
    return And(at_least_one_bw(bool_vars), at_most_one_bw(bool_vars, name)) 

In [2]:
from z3 import *
import time
import numpy as np
from itertools import combinations, product

n=6
start_time = time.time()
weeks = n - 1
periods = n // 2
teams = list(range(n))
home = np.empty((n, weeks, periods), dtype=object)
away = np.empty((n, weeks, periods), dtype=object)

def sports_scheduling_sat(n, timeout=300):
    

    # Define variables
    
    for t in teams:
        for w in range(weeks):
            for p in range(periods):
                home[t, w, p] = Bool(f"H_{t}_{w}_{p}")
                away[t, w, p] = Bool(f"A_{t}_{w}_{p}")

    def build_solver(min_home, max_home):
        s = Solver()
        s.set("random_seed", 42)

        # Slot constraints: each slot has one home and one away
        for w in range(weeks):
            for p in range(periods):
                s.add(exactly_one_seq([home[t, w, p] for t in teams], name=f"slot_home_{w}_{p}"))
                s.add(exactly_one_seq([away[t, w, p] for t in teams], name=f"slot_away_{w}_{p}"))

        # Weekly constraints: each team plays once per week
        for t in teams:
            for w in range(weeks):
                vars_in_week = [home[t, w, p] for p in range(periods)] + [away[t, w, p] for p in range(periods)]
                s.add(exactly_one_seq(vars_in_week, name=f"team_plays_{t}_{w}"))

        # Each pair meets exactly once
        for i, j in combinations(teams, 2):
            match_slots = []
            for w in range(weeks):
                for p in range(periods):
                    match_slots.append(
                        Or(And(home[i, w, p], away[j, w, p]),
                        And(home[j, w, p], away[i, w, p]))
                    )
            s.add(exactly_one_seq(match_slots, name=f"pair_meets_{i}_{j}"))

        # Max 2 games per team per period
        for t in teams:
            for p in range(periods):
                games = [home[t, w, p] for w in range(weeks)] + [away[t, w, p] for w in range(weeks)]
                s.add(at_most_k_seq(games, 2, name=f"team_{t}_period_{p}"))


        # Home/away bounds for fairness
        for t in teams:
            home_games = [home[t, w, p] for w in range(weeks) for p in range(periods)]
            s.add(at_least_k_seq(home_games, min_home, name=f"min_home_{t}"))
            s.add(at_most_k_seq(home_games, max_home, name=f"max_home_{t}"))

        return s

    # Binary search for minimal imbalance
    min_obj = 0
    max_obj = 2 * n - 4
    best_model = None
    best_obj = None

    odd_imbalances = list(range(-(n-1), n, 2))
    #for example, for n=6, odd_imbalances = [-5, -3, -1, 1, 3, 5]

    while min_obj <= max_obj and (time.time() - start_time) < timeout:
        obj = (min_obj + max_obj) // 2
        if obj % 2 == 1:
            obj += 1  # Only even values are valid

        found = False
        print(f"Trying objective value: {obj}")

        # Collect all valid (max_imb, min_imb) pairs for this obj
        max_candidates = []
        min_candidates = []
        for max_imb, min_imb in product(odd_imbalances, repeat=2):
            if max_imb <= min_imb:
                continue
            if max_imb - min_imb != obj:
                continue
            max_candidates.append(max_imb)
            min_candidates.append(min_imb)

        if not max_candidates or not min_candidates:
            min_obj = obj + 2
            continue

        max_imb = max(max_candidates)
        min_imb = min(min_candidates)
        min_home = (n-1 + min_imb) // 2
        max_home = (n-1 + max_imb) // 2
        print(f"  Using max_imb={max_imb}, min_imb={min_imb}, min_home={min_home}, max_home={max_home}")

        s = build_solver(min_home, max_home)
        if s.check() == sat:
            best_model = s.model()
            best_obj = obj
            max_obj = obj - 2
            found = True
        if not found:
            min_obj = obj + 2

    if best_model is None:
        print("No feasible schedule found.")
        return None, time.time() - start_time, None, None

    # Decode the schedule
    sched = [[None for _ in range(weeks)] for _ in range(periods)]
    for w in range(weeks):
        for p in range(periods):
            for i in teams:
                if is_true(best_model.evaluate(home[i, w, p])):
                    for j in teams:
                        if is_true(best_model.evaluate(away[j, w, p])):
                            sched[p][w] = (i + 1, j + 1)

    return sched, time.time() - start_time, best_model, best_obj

# Example usage:
sched, elapsed, model, obj_val = sports_scheduling_sat(6)
print("Optimal obj(N):", obj_val)
print("Elapsed Time:", elapsed, "seconds")
if sched:
    for p, row in enumerate(sched, 1):
        print(f"Period {p}: {row}")

#print for every team the difference between home games and away games
if model:
    for t in range(6):
        home_games = sum(1 for w in range(5) for p in range(3) if is_true(model.evaluate(home[t, w, p])))
        away_games = sum(1 for w in range(5) for p in range(3) if is_true(model.evaluate(away[t, w, p])))
        print(f"Team {t + 1}: Home games = {home_games}, Away games = {away_games}, Difference = {home_games - away_games}")


Trying objective value: 4
  Using max_imb=5, min_imb=-5, min_home=0, max_home=5


NameError: name 'exactly_one_seq' is not defined

In [5]:
from z3 import *
import time
import numpy as np
from itertools import combinations, product

def sports_scheduling_sat(n, timeout=300):
    """
    Solves the sports scheduling problem for n teams using SAT with binary search for minimal imbalance.
    
    The approach:
    1. Possible imbalances are always odd numbers (since games are split into odd-numbered pairs)
    2. The objective is to minimize the difference between max and min imbalance (which must be even)
    3. Uses binary search over possible even objective values to find the minimal feasible one
    """
    start_time = time.time()
    weeks = n - 1
    periods = n // 2
    teams = list(range(n))
    
    # Create variables for home/away assignments
    home = np.empty((n, weeks, periods), dtype=object)
    away = np.empty((n, weeks, periods), dtype=object)
    for t in teams:
        for w in range(weeks):
            for p in range(periods):
                home[t, w, p] = Bool(f"H_{t}_{w}_{p}")
                away[t, w, p] = Bool(f"A_{t}_{w}_{p}")

    def build_solver(min_home=None, max_home=None, min_away=None, max_away=None):
        """Builds a solver with the given home game constraints"""
        s = Solver()
        s.set("random_seed", 42)

        # Each slot has exactly one home and one away team
        for w in range(weeks):
            for p in range(periods):
                s.add(exactly_one_seq([home[t, w, p] for t in teams], name=f"slot_home_{w}_{p}"))
                s.add(exactly_one_seq([away[t, w, p] for t in teams], name=f"slot_away_{w}_{p}"))

        # Each team plays exactly once per week (either home or away)
        for t in teams:
            for w in range(weeks):
                vars_in_week = [home[t, w, p] for p in range(periods)] + [away[t, w, p] for p in range(periods)]
                s.add(exactly_one_seq(vars_in_week, name=f"team_plays_{t}_{w}"))

        # Each pair of teams meets exactly once
        for i, j in combinations(teams, 2):
            match_slots = []
            for w in range(weeks):
                for p in range(periods):
                    match_slots.append(
                        Or(And(home[i, w, p], away[j, w, p]),
                           And(home[j, w, p], away[i, w, p]))
                    )
            s.add(exactly_one_seq(match_slots, name=f"pair_meets_{i}_{j}"))

        # No team plays more than twice in any period
        for t in teams:
            for p in range(periods):
                games = [home[t, w, p] for w in range(weeks)] + [away[t, w, p] for w in range(weeks)]
                s.add(at_most_k_seq(games, 2, name=f"team_{t}_period_{p}"))

        # Home/away game constraints for fairness
        for t in teams:
            home_games = [home[t, w, p] for w in range(weeks) for p in range(periods)]
            s.add(at_least_k_seq(home_games, min_home, name=f"min_home_{t}"))
            s.add(at_most_k_seq(home_games, max_home, name=f"max_home_{t}"))

            #is it necessary to also use the away games constraints?
            #we have to prove that if all teams are constrained to play between, e.g., 1 and 4 home games,
            #there will never be a team that can play 0 or 5 away games, so we can skip this part
            #to prove it mathematically
            """ away_games = [away[t, w, p] for w in range(weeks) for p in range(periods)]
            s.add(at_least_k_seq(away_games, min_away, name=f"min_away_{t}"))
            s.add(at_most_k_seq(away_games, max_away, name=f"max_away_{t}")) """

        num_constrsints = len(s.assertions())
        print(f"Number of constraints: {num_constrsints}")
        return s

    # Binary search setup
    min_obj = 0
    max_obj = 2 * n - 4  # Theoretical maximum difference between max and min imbalance
    
    # All possible odd imbalance values (home - away games difference)
    # For n=6: [-5, -3, -1, 1, 3, 5]
    odd_imbalances = list(range(-(n-1), n, 2))
    
    best_model = None
    best_obj = None
    best_sched = None

    # Binary search over possible even objective values
    while min_obj < max_obj and (time.time() - start_time) < timeout:
        mid = (min_obj + max_obj) // 2
        # Ensure we only test even objective values
        if mid % 2 != 0:
            mid -= 1
            if mid < min_obj:
                min_obj = mid + 2
                continue

        print(f"\nTrying objective value: {mid}")

        # Find all possible (max_imb, min_imb) pairs that satisfy max_imb - min_imb == mid
        # and where max_imb > min_imb and both are odd
        temp_max = float('-inf')
        temp_min = float('inf')
        for max_imb in odd_imbalances:
            for min_imb in odd_imbalances:
                if max_imb >= 0 and min_imb <= 0 and max_imb - min_imb - 2 == mid:
                    temp_max  = max(temp_max, max_imb)
                    temp_min = min(temp_min, min_imb)

        #for n=6, when mid=4, an example of possible_pairs would be:
        # 3 - (-3) - 2
        # 1 -(-5) - 2

        #temp_max is used to find the difference between the at_least(home,min_home) and at_most(home,max_home)
        #min_home+max_home=n-1
        #max_home - min_home = temp_max
        for i in range(n):
            for j in range(i+1, n):
                if i+j == n-1 and j - i == temp_max:
                    max_home=j
                    min_home=i
                if i+j == n-1 and j - i == -temp_min:
                    max_away = j
                    min_away = i
        print(f"  Possible pairs found: max_imb={temp_max}, min_imb={temp_min}")
        print(f'Max_home= {max_home}, Min_home={min_home}')
        print(f'Max_away= {max_away}, Min_away={min_away}')


        # Build and check the solver
        print('I m going to build the solver')
        time_before = time.time()
        s = build_solver(min_home, max_home, min_away, max_away)
        time_after = time.time()
        print(f"  Solver built in {time_after - time_before:.2f} seconds")

        print("  Checking satisfiability...")
        res = s.check()
        time_after2 = time.time()
        print(f"  Check completed in {time_after2 - time_after:.2f} seconds")

        
        if res == sat:
            print("  Found feasible solution")
            best_model = s.model()
            best_obj = mid
            best_sched = [[None for _ in range(weeks)] for _ in range(periods)]
            
            # Decode the schedule
            for w in range(weeks):
                for p in range(periods):
                    for i in teams:
                        if is_true(best_model.evaluate(home[i, w, p])):
                            for j in teams:
                                if is_true(best_model.evaluate(away[j, w, p])):
                                    best_sched[p][w] = (i + 1, j + 1)
            
            # Try to find a better (smaller) objective
            max_obj = mid

        else:
            print("  No solution found, trying higher objective")
            min_obj = mid + 2

    if best_model is None:
        print("\nNo feasible schedule found within time limit.")
        return None, time.time() - start_time, None, None

    # Print team imbalances for verification
    print("\nTeam imbalances:")
    for t in range(n):
        home_games = sum(1 for w in range(weeks) for p in range(periods) if is_true(best_model.evaluate(home[t, w, p])))
        away_games = sum(1 for w in range(weeks) for p in range(periods) if is_true(best_model.evaluate(away[t, w, p])))
        print(f"Team {t + 1}: Home={home_games}, Away={away_games}, Diff={home_games - away_games}")

    return best_sched, time.time() - start_time, best_model, best_obj


# Example usage:
n = 12
sched, elapsed, model, obj_val = sports_scheduling_sat(n)
print("\nOptimal objective value (max imbalance diff):", obj_val)
print("Elapsed Time:", elapsed, "seconds")
if sched:
    weeks = len(sched[0])
    print("        ", end="")
    for w in range(1, weeks+1):
        print(f"Week {w}".center(10), end="")
    print()
    for p, row in enumerate(sched, 1):
        print(f"Period {p}:", end=" ")
        for game in row:
            print(f"{game[0]} v {game[1]}".center(10), end="")
        print()
    print(f"\nExecution time: {elapsed:.2f} seconds")

    # Print home/away counts for each team
    total_imbalance = 0
    for t in range(n):
        home_count = sum(1 for w in range(weeks) for p in range(len(sched)) if sched[p][w] and sched[p][w][0] == t + 1)
        away_count = sum(1 for w in range(weeks) for p in range(len(sched)) if sched[p][w] and sched[p][w][1] == t + 1)
        print(f"Team {t + 1}: Home {home_count}, Away {away_count}")
        total_imbalance += abs(home_count - away_count)
    print(f"Total imbalance: {total_imbalance - n} (should be 0)")


Trying objective value: 10
  Possible pairs found: max_imb=11, min_imb=-11
Max_home= 11, Min_home=0
Max_away= 11, Min_away=0
I m going to build the solver


ArgumentError: argument 1: KeyboardInterrupt: 

#### PROOF THAT IF EVERY TEAM PLAYS BETWEEN MIN_HOME AND MAX_HOME GAMES AT HOME, THEN NO TEAM CAN PLAY FEWER THAN MIN_HOME GAMES AWAY OR MORE THAN MAX_HOME GAMES AWAY

In [17]:
from z3 import *

n = 6         # numero squadre
G = n - 1     # partite totali per squadra

# Crea solver
s = Solver()

# Variabili booleane: home[t][g], away[t][g]
home = [[Bool(f"home_{t}_{g}") for g in range(G)] for t in range(n)]
away = [[Bool(f"away_{t}_{g}") for g in range(G)] for t in range(n)]

# Vincolo 1: ogni partita per squadra t è o in casa o fuori (XOR)
for t in range(n):
    for g in range(G):
        s.add(Xor(home[t][g], away[t][g]))

# Funzione per contare quante variabili sono True in una lista (uso pseudo count con Sum)
def count_true(vars):
    return Sum([If(v, 1, 0) for v in vars])

# Vincolo 2: ogni squadra ha tra 1 e 4 partite in casa
for t in range(n):
    h_count = count_true(home[t])
    s.add(h_count >= 1)
    s.add(h_count <= 4)

# Ora costruiamo la formula OR per "almeno una squadra con 0 o 5 partite away"
away_zero_or_five = []
for t in range(n):
    a_count = count_true(away[t])
    # squadra t ha 0 partite away
    zero_away = (a_count == 0)
    # squadra t ha 5 partite away (cioè tutte)
    five_away = (a_count == G)
    away_zero_or_five.append(Or(zero_away, five_away))

# Aggiungiamo la clausola OR globale: almeno una squadra ha 0 o 5 partite away
s.add(Or(away_zero_or_five))

# Risolvi
result = s.check()
print("Result:", result)
if result == sat:
    print("Esiste almeno una squadra con 0 o 5 partite away (contraddice la tesi).")
    m = s.model()
    # Se vuoi, stampa qualche modello
    for t in range(n):
        h_count = sum([1 for g in range(G) if is_true(m.eval(home[t][g]))])
        a_count = sum([1 for g in range(G) if is_true(m.eval(away[t][g]))])
        print(f"Squadra {t}: home = {h_count}, away = {a_count}")
else:
    print("UNSAT: Nessuna squadra può avere 0 o 5 partite away se tutte hanno tra 1 e 4 partite in casa.")


Result: unsat
UNSAT: Nessuna squadra può avere 0 o 5 partite away se tutte hanno tra 1 e 4 partite in casa.
